# Codes for final project of MUSA620 2019

Wenxin Yang | 2019.05

# 1 Preparation

## 1.1 Load data and packages

### 1.1.1 import packages

In [1]:
import geopandas as gpd
import json
import pandas as pd
import requests
from io import StringIO
from shapely.geometry import Point
import matplotlib.pyplot as plt
from census_area import Census

### 1.1.2 load car crash data

In [2]:
url = 'https://data.boston.gov/dataset/7b29c1b2-7ec2-4023-8292-c24f5d8f0905/resource/e4bfe397-6bfc-49c5-9367-c879fac7401d/download/crash_april_2019.csv.csv'

r = requests.get(url)
df = pd.read_csv(StringIO(r.text))

In [3]:
df['coord'] = list(zip(df.long, df.lat))

df['coord'] = df['coord'].apply(Point)

crash = gpd.GeoDataFrame(df, geometry = 'coord')

crash.crs = ({'init':'epsg:26986'})

In [10]:
crash.head()

,dispatch_ts,mode_type,location_type,street,xstreet1,xstreet2,x_cord,y_cord,lat,long,coord
0,2019-02-28 16:22:25,ped,Street,OLD COLONY TER,SAVIN HILL AVE,WILLIAM T MORRISSEY BLVD,778755.72,2939231.02,42.312350,-71.046643,POINT (-71.04664329961041 42.31234974980679)
1,2019-02-28 16:17:07,mv,Intersection,NaN,DAVID G MUGAR WAY,PINCKNEY ST,771761.61,2956054.41,42.358769,-71.072310,POINT (-71.07231039351991 42.3587691245571)
2,2019-02-28 15:51:51,ped,Street,TERMINAL RD,TERMINAL A,TERMINAL B,785398.37,2958235.76,42.365351,-71.021114,POINT (-71.0211135898958 42.3653507244967)
3,2019-02-28 15:34:55,mv,Street,RADCLIFFE RD,GREENFIELD RD,RUSKINDALE RD,763248.44,2923046.31,42.268228,-71.104232,POINT (-71.1042316375856 42.2682275955996)
4,2019-02-28 14:44:53,ped,Intersection,NaN,BOYLSTON ST,MASSACHUSETTS AVE,767612.72,2951853.40,42.347298,-71.087736,POINT (-71.0877357491058 42.3472975055308)


In [16]:
# convert time of car crash data

crash['dispatch_ts'] = pd.to_datetime(crash['dispatch_ts'], format='%Y-%m-%d %H:%M:%S')
crash['year'] = crash['dispatch_ts'].dt.year
crash['month'] = crash['dispatch_ts'].dt.month
crash['week'] = crash['dispatch_ts'].dt.week

### 1.1.3 load census data

In [4]:
my_api_key = '' # get api key for census data api
api_key = my_api_key
c = Census(key=api_key)

In [5]:
ma_code = 25
boston_code = '07000'

#from https://api.census.gov/data/2017/acs/acs5/variables.html
#B19013_001E is the code for median household income
variables = ('NAME', 'B19013_001E') 
result = c.acs5.state_place(variables, ma_code, 
                            boston_code, year=2017)

In [6]:
inc_tracts = c.acs5.state_place_tract(variables, ma_code, 
                                      boston_code, 
                                      return_geometry=True)
crs = {'init':'epsg:26986'}
inc_df = gpd.GeoDataFrame.from_features(inc_tracts, crs=crs)
len(inc_df)

185

In [7]:
inc_df = inc_df.loc[inc_df['B19013_001E']>0]
len(inc_df)

174

In [84]:
inc_df.head()

,AREALAND,AREAWATER,B19013_001E,BASENAME,CENTLAT,CENTLON,COUNTY,FUNCSTAT,GEOID,INTPTLAT,...,OBJECTID,OID,STATE,STGEOMETRY.AREA,STGEOMETRY.LEN,TRACT,county,geometry,state,tract
0,1795020,0,72478.0,1,+42.3614844,-071.1385888,025,S,25025000100,+42.3614844,...,2146,20755210228163,25,3.289742e+06,11648.544760,000100,025,"POLYGON ((-71.160899 42.358625, -71.160489 42....",25,000100
1,599227,0,80496.0,2.01,+42.3540651,-071.1611678,025,S,25025000201,+42.3540651,...,2147,20755210228189,25,1.097945e+06,5421.299217,000201,025,"POLYGON ((-71.167815 42.353284, -71.1677470000...",25,000201
2,601636,0,72529.0,2.02,+42.3526051,-071.1543443,025,S,25025000202,+42.3526051,...,2148,20755210228214,25,1.102312e+06,5255.261775,000202,025,"POLYGON ((-71.16056500000001 42.352671, -71.16...",25,000202
3,431042,0,50082.0,4.01,+42.3438495,-071.1492929,025,S,25025000401,+42.3438495,...,52753,20755210228237,25,7.895325e+05,3930.695923,000401,025,"POLYGON ((-71.154734 42.341206, -71.1545530000...",25,000401
4,806770,0,69661.0,4.02,+42.3441742,-071.1588288,025,S,25025000402,+42.3441742,...,52754,20755210228285,25,1.477758e+06,4930.417855,000402,025,"POLYGON ((-71.166133 42.340434, -71.166123 42....",25,000402


# 2 Visualizations

## 2.1 Visualizations created with folium

In [9]:
import folium
from folium.plugins import HeatMap

In [12]:
coordcrash = crash[['lat','long']].values

In [75]:
m1 = folium.Map(
    location=[42.31, -71.10],
    tiles='cartodbpositron',
    zoom_start=11
)



HeatMap(coordcrash).add_to(m1)

m1

### 2.1.1 Heatmap with time

In [82]:
def generateBaseMap(default_location=[42.31, -71.10], default_zoom_start=11):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start,tiles='stamentoner')
    return base_map

In [50]:
df_year_list = []

In [65]:
for year in range(2015,2020):
    for month in range(1,13):
        df_year_list.append(crash.loc[(crash['year']==year) & (crash['month']==month)].groupby(['lat','long']).sum().reset_index().values.tolist())

In [66]:
len(df_year_list)

65

In [39]:
from folium.plugins import HeatMap

In [364]:
basemap = generateBaseMap(default_location=[42.31, -71.16],default_zoom_start=11)
HeatMapWithTime(df_year_list,radius=15,gradient={0.2:'blue',0.4:'lime',0.6:'orange',1:'red'},min_opacity=0.6,max_opacity=0.9,use_local_extrema=True).add_to(basemap)

basemap.save('final_heatmapwithtime.html')

In [169]:
ct2 = ct = inc_df[['tract','OBJECTID','B19013_001E','STGEOMETRY.AREA','STGEOMETRY.LEN','geometry']]
ct2.to_file('ct2.geojson',driver='GeoJSON')

In [106]:
import branca

In [305]:
import json
import os
import folium
from folium.plugins import MarkerCluster
MarkerCluster()

In [308]:
colorscale = branca.colormap.linear.YlGnBu_09.scale(10000,220000)

def col(feature):
    inc = feature['properties']['B19013_001E']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if inc is None else colorscale(inc)
        
    }

In [297]:
ctgjson = json.load(open('ct2.geojson'))

### 2.1.2 Choropleth map of median income level & cluster of car crashes

In [362]:

m = folium.Map(
    location=[42.31, -71.10],
    tiles='stamentoner',
    zoom_start=11
)


folium.GeoJson(
    ctgjson,
    name = 'Median Household Income Level',
    style_function = col
).add_to(m)




marker_cluster = MarkerCluster(
    name = 'Cluster of Car Crashes'
).add_to(m)

for point in range(len(crash)):
    folium.Marker(coordcrash[point],
                 tooltip = 'Time of crash: '+str(crash['dispatch_ts'][point]),
                 icon = folium.Icon(
                     color = 'red',
                     icon_color = 'white',
                     icon = 'car',
                     angle = 0,
                     prefix = 'fa'
                 )).add_to(marker_cluster)

folium.LayerControl().add_to(m)



m


m.save('final_cluster_and_choro.html')


## 2.2 Visualizations created with altair

### 2.2.1 Chart of car crashes vs. transit mode & location type

In [331]:
crash.location_type.unique()

array(['Street', 'Intersection', 'Other'], dtype=object)

In [366]:
joined = gpd.sjoin(crash, inc_df, op='within', how='left')

In [333]:
# spatial join

joined1 = gpd.sjoin(crash, inc_df, op='within', how='left').groupby(['year','mode_type','location_type']).size().reset_index()

joined1.columns = ['year','mode_type','location_type','count']

joined1.head()

,year,mode_type,location_type,count
0,2015,bike,Intersection,261
1,2015,bike,Other,10
2,2015,bike,Street,231
3,2015,mv,Intersection,1407
4,2015,mv,Other,334


In [459]:
import altair as alt


pink_blue = alt.Scale(domain=('ped', 'mv','bike'),
                      range=["steelblue", "salmon","orange"])

slider = alt.binding_range(min=2015, max=2019, step=1)
select_year = alt.selection_single(name = 'select',fields=['year'],
                                   bind=slider)

chart1 = alt.Chart(joined1).mark_bar().encode(
    x=alt.X('mode_type:N', title=None),
    y=alt.Y('count:Q', scale=alt.Scale(domain=(0, 2000)),title = 'Number of Crashes'),
    color=alt.Color('mode_type:N', scale=pink_blue),
    column='location_type:N'
).properties(
    width=150
).add_selection(
    select_year
).transform_filter(
    select_year
)


chart1.save('final_count_crash_150.json')

### 2.2.2 Charts of car crashes vs. transit modes / location types over time

In [437]:
crash['date'] = pd.to_datetime(crash['year'].astype(str)+'-'+crash['month'].astype(str))

In [438]:
crash.head()

,dispatch_ts,mode_type,location_type,street,xstreet1,xstreet2,x_cord,y_cord,lat,long,coord,year,month,week,date
0,2019-02-28 16:22:25,ped,Street,OLD COLONY TER,SAVIN HILL AVE,WILLIAM T MORRISSEY BLVD,778755.72,2939231.02,42.312350,-71.046643,POINT (-71.04664329961041 42.31234974980679),2019,2,9,2019-02-01
1,2019-02-28 16:17:07,mv,Intersection,NaN,DAVID G MUGAR WAY,PINCKNEY ST,771761.61,2956054.41,42.358769,-71.072310,POINT (-71.07231039351991 42.3587691245571),2019,2,9,2019-02-01
2,2019-02-28 15:51:51,ped,Street,TERMINAL RD,TERMINAL A,TERMINAL B,785398.37,2958235.76,42.365351,-71.021114,POINT (-71.0211135898958 42.3653507244967),2019,2,9,2019-02-01
3,2019-02-28 15:34:55,mv,Street,RADCLIFFE RD,GREENFIELD RD,RUSKINDALE RD,763248.44,2923046.31,42.268228,-71.104232,POINT (-71.1042316375856 42.2682275955996),2019,2,9,2019-02-01
4,2019-02-28 14:44:53,ped,Intersection,NaN,BOYLSTON ST,MASSACHUSETTS AVE,767612.72,2951853.40,42.347298,-71.087736,POINT (-71.0877357491058 42.3472975055308),2019,2,9,2019-02-01


In [439]:
chart2df = crash.groupby(['date','mode_type']).size().reset_index()

In [449]:
chart3df = crash.groupby(['date','location_type']).size().reset_index()

In [440]:
chart2df.columns = ['date','mode_type','count']

In [452]:
chart3df.columns = ['date','location_type','count']

In [421]:
from datetime import datetime

In [460]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

# The basic line
line = alt.Chart().mark_line().encode(
    alt.X('date:T', axis=alt.Axis(title='')),
    alt.Y('count:Q', axis=alt.Axis(title='',format='f')),
    color='mode_type:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart().mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'count:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart().mark_rule(color='gray').encode(
    x='date:T',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
stockChart = alt.layer(line, selectors, points, rules, text,
                       data=chart2df, 
                       width=500, height=300,title='Car Crashes by Mode Type')


stockChart.save('final_car_crash_mode_time_400.json')

In [461]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

# The basic line
line = alt.Chart().mark_line().encode(
    alt.X('date:T', axis=alt.Axis(title='')),
    alt.Y('count:Q', axis=alt.Axis(title='',format='f')),
    color='location_type:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart().mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'count:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart().mark_rule(color='gray').encode(
    x='date:T',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
loc_type = alt.layer(line, selectors, points, rules, text,
                       data=chart3df, 
                       width=500, height=300,title='Car Crashes by Location Type')


loc_type.save('final_car_crash_location_time_400.json')